In [ ]:
import postgresql

In [35]:
db = postgresql.open('pq://derin:qwerty@localhost:5432/models')

In [ ]:
db.execute("CREATE TABLE model_2_level (number_model serial PRIMARY KEY\
                , ref_to_main_word integer REFERENCES word(number_word)\
                , ref_to_dep_morf integer REFERENCES morf_characters_of_word(number_morf)\
                , imp_feat_main integer REFERENCES important_features(number_imp_feat)\
                , imp_feat_dep integer REFERENCES important_features(number_imp_feat) \
                , prep integer REFERENCES prep(number_prep)\
                , mark real)")

In [ ]:
from Types import *

In [37]:
from Functions import *

In [48]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [38]:
def inputMorf(text = ""):
    print("Выберите характеристики " + text + " морфа\n")
    curMorf = Morf()
    for curEnum in Enums:
        j = 1
        for param in curEnum.__members__:
            print(j, sep = "", end = " - ")
            j += 1
            print(param)
        choose = int(input())
        print("---------------")
        curMorf.__dict__[curEnum.__name__.replace("case", "case1")[1:]] = curEnum(choose)
    print("static: 0 - True, 1 - False\n")
    choose = int(input())
    curMorf.static = (choose == 0)
    return curMorf

In [40]:
def insertModel2(mainWord, depMorf, prep, db1):
    numbMainWord = findWord(mainWord, db1, "главного") #массив номеров главного слова
    numbDepMorf = findMorf(depMorf, db1) #номер зависимого слова
    numbPrep = findPrep(prep, db1)
    for numbCurMain in numbMainWord:
        impMain = inputImpFeat("главного")
        impDep = inputImpFeat("зависимого")
        numbImpMain = findImp(impMain, db1)
        numbImpDep = findImp(impDep, db1)
        end = "WHERE ref_to_main_word = " + str(numbCurMain) + \
            " AND ref_to_dep_morf = " + str(numbDepMorf) + " AND imp_feat_main = " + str(numbImpMain) + \
            " AND imp_feat_dep = " + str(numbImpDep) + " AND prep = " + str(numbPrep)
        s = "SELECT number_model FROM model_2_level " + end
        findUsedModel = db.query(s)#проверка, есть ли уже такая модель
        if (len(findUsedModel) == 0):
            s1 = "INSERT INTO model_2_level VALUES(DEFAULT, " + str(numbCurMain) + ", " + str(numbDepMorf) + ", " + \
            str(numbImpMain) + ", " + str(numbImpDep) + ", " + str(numbPrep) + ", 1)"
            print("\nвставлено\n")
            db.execute(s1)
        else:
            s2 = "UPDATE model_2_level SET mark = mark + 1 " + end
            db.execute(s2)
            print("счетчик увеличен")

In [68]:
insertModel2("ёжиться", parseToMorf("ежом", morph.parse("ежом")[0]), "с", db)

Выберите для данной модели важные свойства для главного слова (укажите в строку через пробел)

0 s_cl_imp
1 animate_imp
2 gender_imp
3 number_imp
4 case1_imp
5 reflection_imp
6 perfective_imp
7 transitive_imp
8 person_imp
9 tense_imp
10 voice_imp
11 degree_imp
12 static_imp
0
Выберите для данной модели важные свойства для зависимого слова (укажите в строку через пробел)

0 s_cl_imp
1 animate_imp
2 gender_imp
3 number_imp
4 case1_imp
5 reflection_imp
6 perfective_imp
7 transitive_imp
8 person_imp
9 tense_imp
10 voice_imp
11 degree_imp
12 static_imp
0 4

вставлено



In [69]:
insertModel2("вознесла", parseToMorf("возницу", morph.parse("возницу")[0]), "None", db)

Выберите для данной модели важные свойства для главного слова (укажите в строку через пробел)

0 s_cl_imp
1 animate_imp
2 gender_imp
3 number_imp
4 case1_imp
5 reflection_imp
6 perfective_imp
7 transitive_imp
8 person_imp
9 tense_imp
10 voice_imp
11 degree_imp
12 static_imp
0 7
Выберите для данной модели важные свойства для зависимого слова (укажите в строку через пробел)

0 s_cl_imp
1 animate_imp
2 gender_imp
3 number_imp
4 case1_imp
5 reflection_imp
6 perfective_imp
7 transitive_imp
8 person_imp
9 tense_imp
10 voice_imp
11 degree_imp
12 static_imp
0 4

вставлено



In [71]:
insertModel2("ёжиться", parseToMorf("быстро", morph.parse("быстро")[0]), "None", db)

Выберите для данной модели важные свойства для главного слова (укажите в строку через пробел)

0 s_cl_imp
1 animate_imp
2 gender_imp
3 number_imp
4 case1_imp
5 reflection_imp
6 perfective_imp
7 transitive_imp
8 person_imp
9 tense_imp
10 voice_imp
11 degree_imp
12 static_imp
0
Выберите для данной модели важные свойства для зависимого слова (укажите в строку через пробел)

0 s_cl_imp
1 animate_imp
2 gender_imp
3 number_imp
4 case1_imp
5 reflection_imp
6 perfective_imp
7 transitive_imp
8 person_imp
9 tense_imp
10 voice_imp
11 degree_imp
12 static_imp
0

вставлено



In [6]:
#видеть(5 cлово, 1 важность) морф(папу(31 морф, 2 важность)), предлога нет(None - 122)
db.execute("INSERT INTO model_2_level VALUES(DEFAULT, 5, 31, 1, 2, 122, 0.5)")

In [7]:
#видеть(5 cлово, 1 важность) морф(маму(32 морф, 2 важность)), предлога нет(None - 122)
db.execute("INSERT INTO model_2_level VALUES(DEFAULT, 5, 32, 1, 2, 122, 0.5)")

In [9]:
db.query("SELECT * FROM model_2_level")

[(1, 5, 31, 1, 2, 122, 0.5), (2, 5, 32, 1, 2, 122, 0.5)]

In [10]:
#любить(6 cлово, 1 важность) морф(папу(31 морф, 2 важность)), предлога нет(None - 122)
db.execute("INSERT INTO model_2_level VALUES(DEFAULT, 6, 31, 1, 2, 122, 0.5)")

In [11]:
#любить(6 cлово, 1 важность) морф(маму(32 морф, 2 важность)), предлога нет(None - 122)
db.execute("INSERT INTO model_2_level VALUES(DEFAULT, 6, 32, 1, 2, 122, 0.5)")

In [12]:
db.query("SELECT * FROM model_2_level")

[(1, 5, 31, 1, 2, 122, 0.5),
 (2, 5, 32, 1, 2, 122, 0.5),
 (3, 6, 31, 1, 2, 122, 0.5),
 (4, 6, 32, 1, 2, 122, 0.5)]

In [8]:
db.execute("INSERT INTO model_2_level VALUES(DEFAULT, 4, 3, 0.6)")

In [9]:
db.query("SELECT * FROM model_2_level")

[(1, 4, 3, 0.6000000238418579)]

In [10]:
db.execute("INSERT INTO model_2_level VALUES(DEFAULT, 4, 6, 0.6)")

In [11]:
db.query("SELECT * FROM model_2_level")

[(1, 4, 3, 0.6000000238418579), (2, 4, 6, 0.6000000238418579)]

In [66]:
db.execute("DROP TABLE model_2_level")